In [2]:
import os, sys

import numpy as np
import pandas as pd
import random

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

from utils.utils import Create_Similarity_Model

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
plt.rc('mathtext', fontset="cm")
import matplotlib.ticker as mtick
import seaborn as sns
import matplotlib

sns.set_style('whitegrid')
sns.set_context("talk")

2022-08-30 10:34:44.139101: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 10:34:44.139118: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
n_nonsimilar = 2
n_similar = 1

In [4]:
possible_re_tau = np.linspace(start = 100, stop = 100000, num = 100)
possible_he = np.linspace(start = 10000, stop = 10000000, num = 100)
possible_r = np.linspace(start = 0.001, stop = 1, num = 100)
Re_tau = []
He_ = []
R_ =[]
U_ = []
dic_x = {}
dic_y = {}

for re_tau in possible_re_tau:
    for r in possible_r:
        for he in possible_he:
            R_.append(r)
            He_.append(he)
            Re_tau.append(re_tau)
            u_ = ((re_tau / 2) * (1 - (r **2))) - ((1 - r) * (he / re_tau))
            U_.append(u_)

c = list(zip(R_, He_, Re_tau, U_))
random.shuffle(c)
R_, He_, Re_tau, U_= zip(*c)

R_ = np.log(R_)
He_ = np.log(He_)
Re_tau = np.log(Re_tau)

dic_x['r^hat'] = R_
dic_x['He'] = He_
dic_x['Re_tau'] = Re_tau
dic_y['u+'] = U_


nonsimilar_keys = ['r^hat', 'He']

xtrain = pd.DataFrame.from_dict(dic_x).values
ytrain = pd.DataFrame.from_dict(dic_y).values

pd.DataFrame.from_dict(dic_x).head()

,r^hat,He,Re_tau
0,-1.096614,15.786050,8.876773
1,-0.106371,13.613728,11.439670
2,-4.501630,15.771885,11.012800
3,-0.420210,14.522650,11.208621
4,-0.856094,15.713130,10.446076


In [81]:
loss = 'mean_squared_error'
optimizer = 'adam'
best_loss = 50.0
n_fits = 5                       # Number of model fits we will try to perform
n_epochs_initial = 10          # Number of epochs we will train inside the loop
n_epochs_final = 10          # Number of epochs we will train after convergence is established
successfull_tries = 0

model = Create_Similarity_Model(n_nonsimilar = n_nonsimilar, n_similar = n_similar)
model.compile(loss = loss, optimizer = optimizer)
model.fit(xtrain, ytrain, epochs = n_epochs_initial, verbose = 1)

# while successfull_tries < n_fits:
#     model = Create_Similarity_Model(n_nonsimilar = n_nonsimilar, n_similar = n_similar)
#     model.compile(loss = loss, optimizer = optimizer)
#     model.fit(xtrain, ytrain, epochs = n_epochs_initial, verbose = 0)
#     current_loss = model.evaluate(xtrain, ytrain, verbose = 0)
#     print("current loss:" + str(current_loss))
#     if current_loss < 70:
#         successfull_tries += 1
#         model.fit(xtrain, ytrain, epochs = n_epochs_final, verbose = 0)
#         good_loss = model.evaluate(xtrain, ytrain, verbose = 0)
#         print(good_loss)
#         if good_loss < best_loss:
#             best_model = model
#             best_loss = good_loss

Epoch 1/10
31250/31250 [==============================] - 36s 1ms/step - loss: 455769152.0000
Epoch 2/10
31250/31250 [==============================] - 33s 1ms/step - loss: 455744640.0000
Epoch 3/10
31250/31250 [==============================] - 32s 1ms/step - loss: 455743712.0000
Epoch 4/10
  334/31250 [..............................] - ETA: 37s - loss: 451604896.0000

KeyboardInterrupt: 

In [70]:
good_loss = model.evaluate(xtrain, ytrain, verbose = 1)

31250/31250 [==============================] - 24s 776us/step - loss: 88393600.0000


In [71]:
for i in range(n_nonsimilar):
    similarity_exponents = model.get_layer('similarity_layer_' + str(i+1)).weights[0][0].numpy()
    print('Exponents multiplicating inside the function for ' + nonsimilar_keys[i] + ': ' + str(similarity_exponents))

multiplication_exponents = model.get_layer('multiplication_layer').weights[0][0].numpy()
print('Exponents multiplicating outside the function: ' + str(multiplication_exponents))

Exponents multiplicating inside the function for r^hat: [-1.2945375]
Exponents multiplicating inside the function for He: [-1.0699544]
Exponents multiplicating outside the function: [0.4752104]
